In [7]:
import numpy as np
import sys
import pandas as pd

from sklearn.metrics import mean_absolute_error

from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU

from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
%matplotlib inline

import scipy

In [8]:
train =pd.read_csv('../data/intern_data.csv').iloc[:,1:]
test = pd.read_csv('../data/intern_test.csv').iloc[:,1:]
num = train.shape[0]

tr_te = pd.concat((train, test), axis = 0)
tr_te_num = pd.get_dummies(tr_te)

In [9]:

tr_te_num  = tr_te_num.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
xtrain,xtest = np.split(tr_te_num.loc[:, tr_te_num.columns != 'y'], [num], axis=0)
xtrain = xtrain.values
xtest = xtest.values
y = train.loc[:,'y']


In [10]:
## neural net
def create_model(num_input):
    model = Sequential()
    
    model.add(Dense(num_input, input_dim = xtrain.shape[1], init = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())
        
    model.add(Dense(220)) 
    model.add(PReLU())
    model.add(BatchNormalization())    
    model.add(Dropout(0.3))
    
    model.add(Dense(60))
    model.add(PReLU())
    model.add(BatchNormalization())    
    model.add(Dropout(0.2))
    
    model.add(Dense(1))
    model.compile(loss = 'mae', optimizer = 'adam')
    return(model)


In [11]:
nfolds = 5
folds = KFold(len(y), n_folds = nfolds, shuffle = True, random_state = 2018)


In [12]:
#early stopping is not helpful, manually tweaking is 
callbacks = [
    EarlyStopping(monitor='val_loss', patience = 30, verbose=0)
]

In [13]:
i = 1
epochs = 260
preds_test = np.zeros(xtest.shape[0])

preds = np.zeros(xtrain.shape[0])
for (inTr, inTe) in folds:
    xtr = xtrain[inTr]
    ytr = y[inTr]
    xte = xtrain[inTe] # xte is xde in fact
    yte = y[inTe]
    pred = np.zeros(xte.shape[0])
    model = create_model(xtr.shape[1])
 
    model.fit(xtr ,ytr,verbose = 0,epochs= epochs, batch_size=32,shuffle= True)
    pred = model.predict(xte)
    score = mean_absolute_error(yte, pred)
    preds[inTe] = pred
    print('Fold ', i, '- MAE:', score)
    i += 1
    
    #predict on the testing set
    preds_test += model.predict(xtest)[:,0]
    
# average and save the predictions on testing set 
preds_test /= nfolds 
test["prediction"] = preds_test
test.to_csv("../output/submission_ANN.csv",index=False)



/home/wxk/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=12, kernel_initializer="he_normal")`
  """


InternalError: Blas GEMM launch failed : a.shape=(32, 12), b.shape=(12, 12), m=32, n=12, k=12
	 [[Node: dense_1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](_arg_dense_1_input_0_1/_231, dense_1/kernel/read)]]
	 [[Node: loss/mul/_299 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2842_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'dense_1/MatMul', defined at:
  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wxk/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  Fil

In [ ]:

plt.scatter(y,preds)

scipy.stats.pearsonr(y,preds)

plt.xlabel('Truth')
plt.ylabel('Prediction')

PCC = round(scipy.stats.pearsonr(y,preds)[0],3)

plt.text(np.min(y)*1.1, 0.95*np.max(preds), r'PCC='+str(PCC),
         fontsize=10)